In [2]:
import numpy as np
import pandas as pd

import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
#import seaborn as sns
import pickle

In [ ]:


THRESHOLD = 500
PEAKS_MAP = dict()

fig = plt.figure()

#df_x_train = pd.read_feather('df_x_train')
#df_test    = pd.read_feather('df_test')
#df_y_train = pd.read_feather('df_y_train')


probs = df_y_train['deal_probability'].values
categories = df_x_train['category_name'].values

print("Total:", probs.shape, categories.shape)
print("Categories:", np.unique(categories))

unique_categories = np.unique(categories)
for cat in unique_categories:
    print(cat, probs[categories == cat].shape)
    cat_probs = probs[categories == cat]
    n, bins, _ = plt.hist(cat_probs, color='blue', bins=100)
    print(n.shape, bins.shape)
    weights = n[n > THRESHOLD]
    n = np.append(n, 1.0)
    peaks = bins[n > THRESHOLD]
    print(weights, peaks)

    PEAKS_MAP[cat] = (peaks, weights)

if True:
    plt.gcf().clear()
    cat_probs = probs[categories == cat]
    n, bins, _ = plt.hist(cat_probs, color='blue', bins=100)
    fig.savefig("output.png")

    n = np.select([n > THRESHOLD, n <= THRESHOLD], [n, n * 0])

    weights = np.ones_like(cat_probs)
    for i in range(cat_probs.shape[0]):
        edge = np.argwhere(bins <= cat_probs[i])[-1][0]
        if edge == 100:
            edge = 99

        if n[edge] == 0:
            weights[i] = 0
    plt.hist(cat_probs, color='red', weights=weights, bins=100)
    fig.savefig("output2.png")

pickle.dump(PEAKS_MAP, open('peaks_map_thresh%d.pkl' % THRESHOLD, 'wb'))

In [3]:
train = pd.read_csv('train.csv')

In [4]:
train['param_2'] = train['param_2'].fillna('unknown')
train['param_1'] = train['param_1'].fillna('unknown')
train['category_name'] = train['category_name'].fillna('unknown')

In [6]:
gp = train.groupby(by=['param_1','param_2','category_name'])['deal_probability'].unique()

In [10]:
gp.head()

param_1     param_2  category_name
ASUS        unknown  Телефоны         [0.0, 0.08401, 0.76786, 0.08603999999999999, 0...
Acer        unknown  Телефоны         [0.76786, 0.15803, 0.0, 0.43177, 0.27148, 0.22...
Alcatel     unknown  Телефоны         [0.0, 0.08693, 0.07851, 0.0899, 0.07074, 0.738...
BQ          unknown  Телефоны         [0.0, 0.12754000000000001, 0.18945, 0.76786, 0...
BlackBerry  unknown  Телефоны         [0.0, 0.05466, 0.19927, 0.76786, 0.08932000000...
Name: deal_probability, dtype: object

In [12]:
gp = gp.reset_index()

In [13]:
gp.head()

,param_1,param_2,category_name,deal_probability
0,ASUS,unknown,Телефоны,"[0.0, 0.08401, 0.76786, 0.08603999999999999, 0..."
1,Acer,unknown,Телефоны,"[0.76786, 0.15803, 0.0, 0.43177, 0.27148, 0.22..."
2,Alcatel,unknown,Телефоны,"[0.0, 0.08693, 0.07851, 0.0899, 0.07074, 0.738..."
3,BQ,unknown,Телефоны,"[0.0, 0.12754000000000001, 0.18945, 0.76786, 0..."
4,BlackBerry,unknown,Телефоны,"[0.0, 0.05466, 0.19927, 0.76786, 0.08932000000..."


In [14]:
gp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 808 entries, 0 to 807
Data columns (total 4 columns):
param_1             808 non-null object
param_2             808 non-null object
category_name       808 non-null object
deal_probability    808 non-null object
dtypes: object(4)
memory usage: 25.3+ KB


In [18]:
0 * 1

0

In [19]:
exact_bins = []
for idx, c_row in gp.iterrows():
    length = c_row['deal_probability'].shape[0]
    deal_probability_a = c_row['deal_probability']
    tmp_bin = np.nan
    for i in range(length, length+10):
        total_set = set(np.linspace(0.0,1.0, num=i))
        trigger = 1
        for dprob in deal_probability_a:
            if(dprob in total_set):
                trigger = trigger * 1
            else:
                trigger = trigger * 0
        if(trigger==1):
            tmp_bin = i
            break
    exact_bins.append(tmp_bin)
        
        
    

In [20]:
gp['exact_bins'] = exact_bins
gp.head()

,param_1,param_2,category_name,deal_probability,exact_bins
0,ASUS,unknown,Телефоны,"[0.0, 0.08401, 0.76786, 0.08603999999999999, 0...",NaN
1,Acer,unknown,Телефоны,"[0.76786, 0.15803, 0.0, 0.43177, 0.27148, 0.22...",NaN
2,Alcatel,unknown,Телефоны,"[0.0, 0.08693, 0.07851, 0.0899, 0.07074, 0.738...",NaN
3,BQ,unknown,Телефоны,"[0.0, 0.12754000000000001, 0.18945, 0.76786, 0...",NaN
4,BlackBerry,unknown,Телефоны,"[0.0, 0.05466, 0.19927, 0.76786, 0.08932000000...",NaN


In [24]:
gp.to_csv('quantum.csv', index=False)

In [23]:
gp[gp['exact_bins']!='']

/home/dcek/anaconda3/lib/python3.6/site-packages/pandas/core/ops.py:816: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(x, name)(y)


TypeError: invalid type comparison

In [ ]:
gp['exact_bins']

In [16]:
len(gp)

808